In [ ]:
# Install Java, Spark, and Findspark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q -c http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvidia.com/comp

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-13 22:14:14--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  3.92MB/s    in 0.2s    

2020-08-13 22:14:14 (3.92 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:

# Start Spark session
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# from pyspark import SparkFiles
# #url = "https://drive.google.com/file/d/1TYor4zGC30w8zGPXa9l0L0bc447wVMOX/view?usp=sharing"
# from google.colab import drive
# drive.mount('/gdrive',force_remount=True)
# import glob

# url = glob.glob("/gdrive/My Drive/")
#https://data-boogtcamp-luminda.s3.us-east-2.amazonaws.com/amazon_reviews_us_Toys_v1_00.tsv
from pyspark import SparkFiles
url="https://data-boogtcamp-luminda.s3.us-east-2.amazonaws.com/amazon_reviews_us_Toys_v1_00.tsv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
user_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!|2015-08-31 00:00:00|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...| 

In [ ]:
user_data_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
print(user_data_df.count())

4864249


In [ ]:
print(len(user_data_df.columns))


15


In [ ]:
df = user_data_df.dropna()
print(df.count())

4863497


In [ ]:
# mode = "append"
# jdbc_url="jdbc:postgresql://http://127.0.0.1:53270/browser/creditcard"
# config = {"user":"postgres", 
#           "password": "postgres", 
#           "driver":"org.postgresql.Driver"}
#toysdatabase.c6cphbc1v9gp.us-east-2.rds.amazonaws.com

mode = "append"
jdbc_url="jdbc:postgresql://toysdatabase.c6cphbc1v9gp.us-east-2.rds.amazonaws.com:5432/toysdatabase"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to active_user table in RDS

df.write.jdbc(url=jdbc_url, table='toys', mode=mode, properties=config)
